# Wykład 11 - Wielowątkowość - Wstęp

## 11.1 Klasa `Thread` i interfejs `Runnable`

In [5]:
// java
public class A extends Thread {
  public static void main() {
    A thread = new A();
    thread.start();
    System.out.println("poza wątkiem");
  }
  public void run() {
    System.out.println("wewnątrz wątku");
  }
}
A.main();

poza wątkiem
wewnątrz wątku


In [7]:
// java
public class B implements Runnable {
  public static void main() {
    B b = new B();
    Thread thread = new Thread(b);
    thread.start();
    System.out.println("poza wątkiem");
  }
  public void run() {
    System.out.println("wewnątrz wątku");
  }
}
B.main();

poza wątkiem
wewnątrz wątku


In [6]:
// kotlin
class C : Thread(){
    override fun run(){
        println("wewnątrz wątki")
    }
}

fun main(){
    val thread = C()
    thread.start()
    println("poza wątkiem")
}

main()

poza wątkiem
wewnątrz wątki


In [11]:
// kotlin
class D: Runnable{
    override fun run(){
        println("wewnątrz wątki")
    }
}

fun main(){
    val d = D()
    val thread = Thread(d)
    thread.start()
    println("poza wątkiem")
}

main()

poza wątkiem
wewnątrz wątki


## 11.2 Współbieżność

In [18]:
// java
public class E extends Thread {
  public static int counter = 0;

  public static void main() {
    E thread = new E();
    thread.start();
    // try{
    //     Thread.sleep(2000);
    // } catch (InterruptedException e){}
    System.out.println(counter);
    counter++;
    System.out.println(counter);
  }

  public void run() {
    counter++;
  }
}
E.main();

0
2


## 11.3 Widoczność

In [21]:
public class VisibilityDemoMain {
    private static int sCounter = 0;
    //private static volaile int sCounter = 0;

    public static void main(){
        new Consumer().start();
        try {
            Thread.sleep(500);
        } catch (InterruptedException e) {
            return;
        }
        new Producer().start();
    }

    static class Consumer extends Thread {
        @Override
        public void run() {
            int localCounter = -1;
            while (true){
                if(localCounter != sCounter){
                    System.out.println("Consumer: " + sCounter);
                    localCounter = sCounter;
                }
                if(sCounter >= 7)
                    break;
            }
            System.out.println("Consumer is terminated");
        }
    }

    static class Producer extends Thread{
        @Override
        public void run() {
            while(sCounter < 7){
                int localValue = sCounter;
                localValue++;
                System.out.println("Producer: " + localValue);
                sCounter = localValue;
                try {
                    Thread.sleep(1000);
                } catch (InterruptedException e) {
                    return;
                }
            }
            System.out.println("Producer is terminated");
        }
    }
}

VisibilityDemoMain.main();

Consumer: 7
Consumer is terminated
Producer is terminated


Wątek `Consumer` wykrywa pierwszą zmianę i drukuje wiadomość. Następnie pracę rozpoczyna `Producer` i drukuje kolejne wiadomości. Tutaj nie widzimy dalszych wiadomości od `Consumer`, dzieje się tak ponieważ wątek ten nie wykrywa zmian wartości `sCounter`.

Problem powoduje model pamięci zastosowany w **JVM** - wartość `sCounter` jest przechowywana w pamięci podręcznej, więc pomimo tego że `Producer` modyfikuje jego wartość, `Consumer` nie jest w stanie się do tego elementu pamięci dostać. Zastosowana optymalizacja powoduje utworzenie kopii w pamięci podręcznej która cały czas ma wartość 0.

Taki stan możemy rozwiązać wprowadzając **zmienną ulotną** z modyfikatorem `volatile` (w Kotlinie `@Volatile`). Powoduje on że wartość zmiennej zawsze jest odczytywany z pamięci i nigdy nie jest przechowywana w pamięci podręcznej.

In [ ]:
fun main() {
    Consumer().start()
    try {
        Thread.sleep(500)
    } catch (e: InterruptedException) {
        return
    }
    Producer().start()
}

private var sCounter = 0
// @Volatile private var sCounter = 0

class Producer : Thread() {
    override fun run() {
        while (sCounter < 7) {
            var localValue = sCounter
            localValue++
            println("Producer: $localValue")
            sCounter = localValue
            try {
                sleep(1000)
            } catch (e: InterruptedException) {
                return
            }
        }
        println("Producer is terminated")
    }
}

class Consumer : Thread() {
    override fun run() {
        var localCounter = -1
        while (true) {
            if (localCounter != sCounter) {
                println("Consumer: $sCounter")
                localCounter = sCounter
            }
            if (sCounter >= 7) break
        }
        println("Consumer is terminated")
    }
}


## 11.4 Klasy atomowe

In [ ]:
// java
public class Main {
    private static final int COUNT_UP_TO = 1000;
    private static final int NUM_OF_THREADS = 100;

    private static volatile int mCount;

    public static void main(String[] args) throws InterruptedException {
        mCount.set(0);

        for(int i = 0; i < NUM_OF_THREADS; i++){
            startThread();
        }
        Thread.sleep(2000);
        System.out.println(mCount.get());
    }

    private static void startThread() {
        new Thread(() ->{
            for(int i = 0; i < COUNT_UP_TO; i++){
                mCount++;
            }
        }).start();
    }
}

In [ ]:
// kotlin
private const val COUNT_UP_TO = 1000
private const val NUM_OF_THREADS = 100

@Volatile
private var mCount = 0

fun main() {
    mCount = 0
    for (i in 0 until NUM_OF_THREADS) {
        startThread()
    }
    Thread.sleep(2000)
    println(mCount)
}

private fun startThread() {
    Thread {
        for (i in 0 until COUNT_UP_TO) {
            mCount++
        }
    }.start()
}

Mamy 100 wątków liczących do 1000, więc spodziewamy się że wartość `mCount` będzie wynosić 100000 po wykonaniu programu. Jednak jak widzimy wartość często jest mniejsza. Umieszczając powyższy kod w IDE powinniśmy dostać ostrzeżenie **Non-atomic operation on volatile field**.

Operacją nie-atomową jest przykładowo inkrementacja `mCount++`. Inkrementacja w rzeczwistości jest wykonywana w trzech krokach.

```java
int localCopy = mCount;
mCount = localCopy + 1;
```

- tworzona jest kopia `mCount`
- wykonywana jest operacja dodawania `localCopy + 1`
- wykonywany jest zapis do zmiennej `mCount`

Gdy wiele wątków zaczyna przeprowadzać tę operację współbieżnie istnieje możliwość odczytania wartości `mCount` zanim zostanie wykonana operacja dodawania i powrotnego przypisania wartości.

Chcąc zapewnić bezpieczeństwo podczas inkrementacji zmiennej `mCount` przez wiele wątków musimy zastosować zmienną typu atomowego.

In [ ]:
//java
import java.util.concurrent.atomic.AtomicInteger;

public class Main {
    private static final int COUNT_UP_TO = 1000;
    private static final int NUM_OF_THREADS = 100;

    private static final AtomicInteger mCount = new AtomicInteger(0);
    //private static int mCount = 0;

    public static void main(String[] args) throws InterruptedException {
        //mCount=0;
        mCount.set(0);

        for(int i = 0; i < NUM_OF_THREADS; i++){
            startThread();
        }
        Thread.sleep(2000);
        //System.out.println(mCount);
        System.out.println(mCount.get());
    }

    private static void startThread() {
        new Thread(() ->{
            for(int i = 0; i < COUNT_UP_TO; i++){
                //mCount++;
                mCount.incrementAndGet();
            }
        }).start();
    }
}

In [ ]:
//kotlin
import java.util.concurrent.atomic.AtomicInteger

private const val COUNT_UP_TO = 1000
private const val NUM_OF_THREADS = 100

//@Volatile
//private var mCount = 0

private val mCount = AtomicInteger(0)

fun main() {
    //mCount = 0
    mCount.set(0)
    for (i in 0 until NUM_OF_THREADS) {
        startThread()
    }
    Thread.sleep(2000)
    //println(mCount)
    println(mCount.get())
}

private fun startThread() {
    Thread {
        for (i in 0 until COUNT_UP_TO) {
            //mCount++
            mCount.incrementAndGet()
        }
    }.start()
}

## 11.5 Synchronizacja

```java
static class Producer extends Thread {
        @Override
        public void run() {
            while (true) {

                    if(sCounter >= 7)
                        break;
                    int localValue = sCounter;
                    localValue++;
                    System.out.println("Producer: " + localValue);
                    sCounter = localValue;
                

                // try {
                //     Thread.sleep(1000);
                // } catch (InterruptedException e) {
                //     return;
                // }
            }
            System.out.println("Producer is terminated");
        }
    }
```

Jeżeli pozbędziemy się wywołania `Thread.sleep` z klasy `Producer` wystąpi **race condition**. Dwa lub więcej wątków w sposób niekontrolowany uzyskują dostęp do tej samej informacji.

Aby rozwiązać ten problem musimy zapewnić atomowość dostępu do danych. Gdy któryś z wątków wchodzi w pętle `while` chcemy zapewnić bezpieczeństwo wykonania - czyli uniemożliwić innemu wątkowi pracę na współdzielonych danych. Osiągamy to przez **synchronizację** pracy wątków.

In [ ]:
public class Main {
    private static volatile int sCounter = 0;

    private static final Object LOCK = new Object();

    public static void main(String[] args) {
        new Consumer().start();
        try {
            Thread.sleep(500);
        } catch (InterruptedException e) {
            return;
        }
        new Producer().start();
    }

    static class Consumer extends Thread {
        @Override
        public void run() {
            int localCounter = -1;
            while (true) {
                synchronized (LOCK) {
                    if (localCounter != sCounter) {
                        System.out.println("Consumer: " + sCounter);
                        localCounter = sCounter;
                    }
                    if (sCounter >= 7)
                        break;
                }
            }
            System.out.println("Consumer is terminated");
        }
    }

    static class Producer extends Thread {
        @Override
        public void run() {
            while (true) {
                synchronized (LOCK) {
                    if(sCounter >= 7)
                        break;
                    int localValue = sCounter;
                    localValue++;
                    System.out.println("Producer: " + localValue);
                    sCounter = localValue;
                }
            }
            System.out.println("Producer is terminated");
        }
    }
}


In [ ]:
public class Main {
    private static int sCounter = 0;

    private static final Object LOCK = new Object();

    public static void main(String[] args) {
        new Consumer().start();
        try {
            Thread.sleep(500);
        } catch (InterruptedException e) {
            return;
        }
        new Producer().start();
    }

    static class Consumer extends Thread {
        @Override
        public void run() {
            int localCounter = -1;
            while (true) {
                synchronized (LOCK) {
                    if (localCounter != sCounter) {
                        System.out.println("Consumer: " + sCounter);
                        localCounter = sCounter;
                    }
                    if (sCounter >= 7)
                        break;
                    
                    LOCK.notifyAll();
                    
                    try {
                        LOCK.wait();
                    } catch (InterruptedException e) {
                        return;
                    }
                }

            }
            System.out.println("Consumer is terminated");
        }
    }

    static class Producer extends Thread {
        @Override
        public void run() {
            while (true) {
                synchronized (LOCK) {
                    if(sCounter >= 7)
                        break;
                    int localValue = sCounter;
                    localValue++;
                    System.out.println("Producer: " + localValue);
                    sCounter = localValue;
                    
                    LOCK.notifyAll();
                    
                    try {
                        LOCK.wait();
                    } catch (InterruptedException e) {
                        return;
                    }

                }
            }
            System.out.println("Producer is terminated");
        }
    }
}
